In [1]:
import pickle
import pandas as pd
import warnings
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import optimize
import pickle
import statsmodels.api as sm
from scipy import stats
import matplotlib.patches as mpatches 
from cycler import cycler

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [1]:
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(100,100))
%matplotlib inline
shp_path = "./Health_Districts__2022_.shp"
sf = shp.Reader(shp_path)
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df
df = read_shapefile(sf)


In [2]:
df

,OBJECTID,HD,HD_NAME,SPA,SPA_NAME,HD_NUM,SHAPE_Leng,SHAPE_Area,coords
0,1,3,Alhambra,3,San Gabriel,3,2.013738e+05,1.070967e+09,"[(6529037.730845809, 1832223.2820242345), (652..."
1,2,5,Antelope Valley,1,Antelope Valley,5,1.441314e+06,-3.787690e+10,"[(6665669.1229368, 1967117.0561993867), (66646..."
2,3,6,Bellflower,7,East,6,2.633842e+05,-1.095872e+09,"[(6508783.824175224, 1756527.2585478127), (650..."
3,4,9,Central,4,Metro,9,1.812114e+05,9.041297e+08,"[(6489022.93718788, 1827887.899290815), (64890..."
4,5,12,Compton,6,South,12,1.648679e+05,7.024756e+08,"[(6486177.461579636, 1775642.3518792242), (648..."
5,6,16,East LA,7,East,16,1.479064e+05,6.508497e+08,"[(6507733.016565546, 1820814.4327947348), (650..."
6,7,19,East Valley,2,San Fernando,19,3.212520e+05,2.055956e+09,"[(6457937.2844643, 1864927.926916644), (645791..."
7,8,23,El Monte,3,San Gabriel,23,2.973065e+05,1.732918e+09,"[(6587394.883488223, 1810820.3143315613), (658..."
8,9,25,Foothill,3,San Gabriel,25,1.067696e+06,-1.856290e+10,"[(6665669.1229368, 1967117.0561993867), (66656..."
9,10,27,Glendale,2,San Fernando,27,2.490039e+05,1.673982e+09,"[(6484710.135446057, 1865698.9076581448), (648..."


In [3]:
districts = {'Antelope Valley':0, 
            'East Valley':0, 
            'Glendale':0, 
            'San Fernando':2, 
            'West Valley':3,
            'Alhambra':0, 
            'El Monte':3, 
            'Foothill':0, 
            'Pasadena':0, 
            'Pomona':0,
             'Central':2, 
             'Hollywood-Wilshire':0, 
             'Northeast':1, 
             'West':0,
             'Compton':0, 
             'South':0, 
             'Southeast':0, 
             'Southwest':1,
             'Bellflower':3, 
             'East LA':0, 
             'San Antonio':1, 
             'Whittier':0,
             'Harbor':0, 
             'Inglewood':2, 
             'Long Beach':0, 
             'Torrance':0}

In [4]:
id_map = {}
for i in districts:
    id_map[df[df['HD_NAME']==i].index[0]]=districts[i]

In [5]:
count = 0
for i in districts:
    districts[i] = count
    count+=1

In [ ]:
Fonts = 8
def plot_comunas_data(sf, title, comunas, data=None, 
                      color=None, print_id=False):
    '''
    Plot map with selected comunes, using specific color
    '''
    
    df = read_shapefile(sf)
    comuna_id = []
    for i in comunas:
        if i not in df.HD_NAME.values:
            continue
        comuna_id.append(df[df['HD_NAME']==i].index[0])
    plot_map_fill_multiples_ids_tone(sf, title, comuna_id, 
                                     print_id, 
                                     figsize = (11,9));
def plot_map_fill_multiples_ids_tone(sf, title, comuna,  
                                     print_id, 
                                     figsize = (11,9)):
    '''
    Plot map with lim coordinates
    '''
        
    plt.figure(figsize = figsize)
    fig, ax = plt.subplots(figsize = figsize,dpi=1200)
    #fig.suptitle(title, fontsize=16)
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.plot(x, y, 'k')
    for id in comuna:
        shape_ex = sf.shape(id)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        if id_map[id] == 0:
            filled_poly = ax.fill(x_lon,y_lat,'mintcream')
            #print(x_lon)
            if df['HD_NAME'][id] == 'Antelope Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.25))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'East Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.max(y_lat)+np.quantile(y_lat,0.6))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.3))/2,(np.max(y_lat)+np.quantile(y_lat,0.45))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.73))/2,(np.max(y_lat)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Hollywood-Wilshire':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.8)+np.quantile(y_lat,0))/2, 'Hollywood-\nWilshire', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Pasadena':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.7)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Central':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Northeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.1))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'North-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.15))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'South-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southwest':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0.3))/2, 'South-\nwest', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Foothill':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Harbor':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.7))/2,1730000, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            else:
                ax.text((np.max(x_lon)+np.min(x_lon))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
        elif id_map[id] == 2:
            filled_poly = ax.fill(x_lon,y_lat,'olive')
            #print(x_lon)
            if df['HD_NAME'][id] == 'Antelope Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.25))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'East Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.max(y_lat)+np.quantile(y_lat,0.6))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.3))/2,(np.max(y_lat)+np.quantile(y_lat,0.45))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.73))/2,(np.max(y_lat)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Hollywood-Wilshire':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.8)+np.quantile(y_lat,0))/2, 'Hollywood-\nWilshire', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Pasadena':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.7)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Central':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Northeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.1))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'North-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.15))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'South-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southwest':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0.3))/2, 'South-\nwest', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Foothill':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Harbor':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.7))/2,1730000, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            else:
                ax.text((np.max(x_lon)+np.min(x_lon))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
        elif id_map[id] == 3:
            filled_poly = ax.fill(x_lon,y_lat,'yellowgreen')
            #print(x_lon)
            if df['HD_NAME'][id] == 'Antelope Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.25))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'East Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.max(y_lat)+np.quantile(y_lat,0.6))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.3))/2,(np.max(y_lat)+np.quantile(y_lat,0.45))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.73))/2,(np.max(y_lat)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Hollywood-Wilshire':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.8)+np.quantile(y_lat,0))/2, 'Hollywood-\nWilshire', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Pasadena':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.7)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Central':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Northeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.1))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'North-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.15))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'South-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southwest':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0.3))/2, 'South-\nwest', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Foothill':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Harbor':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.7))/2,1730000, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            else:
                ax.text((np.max(x_lon)+np.min(x_lon))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
        else:
            ax.fill(x_lon,y_lat, 'salmon')
            if df['HD_NAME'][id] == 'Antelope Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.25))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'East Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.max(y_lat)+np.quantile(y_lat,0.6))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West Valley':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.3))/2,(np.max(y_lat)+np.quantile(y_lat,0.45))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'West':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.73))/2,(np.max(y_lat)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Hollywood-Wilshire':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.8)+np.quantile(y_lat,0))/2, 'Hollywood-\nWilshire', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Pasadena':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,0.7)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Central':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Northeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.1))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'North-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southeast':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.15))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, 'South-\neast', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Southwest':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0.3))/2, 'South-\nwest', ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Foothill':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.4))/2,(np.quantile(y_lat,1)+np.quantile(y_lat,0))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            elif df['HD_NAME'][id] == 'Harbor':
                ax.text((np.max(x_lon)+np.quantile(x_lon,0.7))/2,1730000, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
            else:
                ax.text((np.max(x_lon)+np.min(x_lon))/2,(np.max(y_lat)+np.min(y_lat))/2, df['HD_NAME'][id], ha='center', va='center', fontsize=Fonts, color='black')
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        if print_id != False:
            x0 = np.mean(x_lon)
            y0 = np.mean(y_lat)
            plt.text(x0, y0, id, fontsize=10)
    #plt.xlim(-119,-117.6)
    plt.ylim(1714000, 2130000)
    pop_a = mpatches.Patch(color='yellowgreen', label='Recommended by model') 
    pop_b = mpatches.Patch(color='salmon', label='Recommended by actual') 
    pop_c = mpatches.Patch(color='olive', label='Recommended by both') 
    plt.legend(handles=[pop_a,pop_b,pop_c],fontsize=14,loc = 'lower left') 
    fig.savefig('spa_model.pdf', format='pdf', dpi=1200, bbox_inches="tight")
print_id = False # The shape id will be printed
color_pallete = 0 # 'Purples'
south = list(districts.keys())
datax = list(districts.values())
plot_comunas_data(sf, 'Traffic Increase', south, datax, color_pallete, print_id)

<Figure size 1100x900 with 0 Axes>

In [ ]:
np.quantile